### Imports


In [8]:
%matplotlib inline
import matplotlib.pyplot as plt
import math
from collections import deque
from pylab import rcParams
import numpy as np

### Graph class

In [9]:
class Graph(object):
    '''Represents a graph'''

    def __init__(self, vertices, edges):
        '''A Graph is defined by its set of vertices
           and its set of edges.'''
        self.V = set(vertices) # The set of vertices
        self.E = set([])       # The set of edges
        self.neighbors = {}          # A dictionary that will hold the list
                               # of adjacent vertices for each vertex.
        self.vertex_coordinates = {}       # An optional dictionary that can hold coordinates
                               # for the vertices.
        self.edge_labels = {}  # a dictionary of labels for edges

        self.add_edges(edges)  # Note the call to add_edges will also
                               # update the neighbors dictionary
        print ('(Initializing a graph with %d vertices and %d edges)' % (len(self.V),len(self.E)))


    def add_vertices(self,vertex_list):
        ''' This method will add the vertices in the vertex_list
            to the set of vertices for this graph. Since V is a
            set, duplicate vertices will not be added to V. '''
        for v in vertex_list:
            self.V.add(v)
        self.build_neighbors()


    def add_edges(self,edge_list):
        ''' This method will add a list of edges to the graph
            It will insure that the vertices of each edge are
            included in the set of vertices (and not duplicated).
            It will also insure that edges are added to the
            list of edges and not duplicated. '''
        for s,t in edge_list:
            if (s,t) not in self.E and (t,s) not in self.E:
                self.V.add(s)
                self.V.add(t)
                self.E.add((s,t))
        self.build_neighbors()


    def build_neighbors(self):
        self.neighbors = {}
        for v in self.V:
            self.neighbors[v] = []
        for e in self.E:
            s,t = e
            self.neighbors[s].append(t)
            self.neighbors[t].append(s)

### Network class

In [10]:
class Network(Graph):    
    def __init__(self, vertices, edge_weights):
        ''' Initialize the network with a list of vertices
        and weights (a dictionary with keys (E1, E2) and values are the weights)'''

        edges = []
        for e1,e2 in edge_weights:
            edges.append((e1,e2))
        
        Graph.__init__(self, vertices, edges)
        self.weights = {}
        for e1,e2 in edge_weights:
            weight = edge_weights[(e1,e2)]
            self.weights[(e1,e2)] = weight
            self.weights[(e2,e1)] = weight
        self.edge_labels = self.weights

### Bellman Ford naive implementation

In [19]:
def BellmanFord(network, source):
    
    #Number of vertices
    vertCount = len(network.V)
    
    #Initialize distance to infinity and source distance to 0
    distance = [np.inf] * vertCount
    distance[source] = 0
    
    #Check every vertix and relax nodes V - 1 times
    for i in range(vertCount - 1):
        for e in network.E:
            u,v = e
            if not np.isinf(distance[u]) and distance[v] > distance[u] + network.weights[(u,v)]:
                distance[v] = distance[u] + network.weights[(u,v)]
    
    #Negative Cycle check
    #Any update after V - 1 iterations indicate negative cycles
    for e in network.E:
        u,v = e
        if not np.isinf(distance[u]) and distance[v] > distance[u] + network.weights[(u,v)]:
            print("\nNegative Cycle Detected\n")
            return distance, vertCount
        
#     printSol(distance, vertCount)
    return distance, vertCount
    
def printSol(distance, count):
    """Prints the shortest distance from source to all otehr vertices in the graph"""
    print("\nVertix distance from source")
    print("----------------------------")
    print("Vertix \t \t Distance")
    print("------ \t \t --------")
    for i in range(count):
        print("  %c \t\t   %d" % ( i+65, distance[i]))

### Test case 1 : Geeks for geeks dataset

In [29]:
V = [0, 1, 2, 3, 4]
# W = {(0, 2):3, (0, 1):6, (0, 3):7, (1, 2):1, (1, 3):2, (2, 4):10, (1, 4):4}
W = {(0, 1):-1, (0, 2):4, (1,2):3, (1,3):2, (1,4):2, (3,2):5, (3,1):1, (4,3):-3}
G1 = Network(V,W)
print (G1.weights)

distance, count = BellmanFord(G1,0)
printSol(distance,count)

(Initializing a graph with 5 vertices and 7 edges)
{(0, 1): -1, (1, 0): -1, (0, 2): 4, (2, 0): 4, (1, 2): 3, (2, 1): 3, (1, 3): 1, (3, 1): 1, (1, 4): 2, (4, 1): 2, (3, 2): 5, (2, 3): 5, (4, 3): -3, (3, 4): -3}

Vertix distance from source
----------------------------
Vertix 	 	 Distance
------ 	 	 --------
  A 		   0
  B 		   -1
  C 		   2
  D 		   -2
  E 		   1


### Test case 2: Negative Cycles

In [18]:
V = [0, 1, 2, 3]
W = {(0, 1):5, (0, 2):4, (1,3):3, (2,1):-6, (3,2):2}
G2 = Network(V,W)
print (G2.weights)

distance, count = BellmanFord(G2,0)
printSol(distance,count)

(Initializing a graph with 4 vertices and 5 edges)
{(0, 1): 5, (1, 0): 5, (0, 2): 4, (2, 0): 4, (1, 3): 3, (3, 1): 3, (2, 1): -6, (1, 2): -6, (3, 2): 2, (2, 3): 2}

Negative Cycle Detected

Vertix distance from source
----------------------------
Vertix 	 	 Distance
------ 	 	 --------
  A 		   0
  B 		   -2
  C 		   4
  D 		   1


### SPFA FIFO

In [27]:
def SPFA_FIFO(network, source):
    """SPFA with FIFO queues"""
    #vertices count
    vertCount = len(network.V)
    
    #Flag to see if queue is empty
    flag = True
    
    #Initialize distance to infinity and source distance to 0
    distance = [np.inf] * vertCount
    distance[source] = 0
    
    #Initialize the visited node array
    visited = [0] * vertCount
    
    #A doubly ended queue to store visited vertices
    Q = deque();
    
    #Initialize the deque by adding the source node
    Q.append(source)
    
    #Run as long as the queue is not empty
    while(flag):
        
        x = Q.popleft()
        
        for i in network.neighbors[x]:
            visited[i] = visited[i] + 1 #Visited count
            if(visited[i] > vertCount - 1):
                print("\nNegative Cycles Found\n")
                return distance, vertCount
            else:
                if distance[i] > distance[x] + network.weights[(x,i)]:
                    distance[i] = distance[x] + network.weights[(x,i)]
                    if not(i in Q):
                        Q.append(i)
        if(Q):
            flag = 1 #Not empty
        else:
            flag = 0 #Empty
    
    return distance, vertCount

In [28]:
V = [0, 1, 2, 3, 4]
# W = {(0, 2):3, (0, 1):6, (0, 3):7, (1, 2):1, (1, 3):2, (2, 4):10, (1, 4):4}
W = {(0, 1):-1, (0, 2):4, (1,2):3, (1,3):2, (1,4):2, (3,2):5, (3,1):1, (4,3):-3}
G1 = Network(V,W)
print (G1.weights)

distance, count = SPFA_FIFO(G1,0)
printSol(distance,count)

(Initializing a graph with 5 vertices and 7 edges)
{(0, 1): -1, (1, 0): -1, (0, 2): 4, (2, 0): 4, (1, 2): 3, (2, 1): 3, (1, 3): 1, (3, 1): 1, (1, 4): 2, (4, 1): 2, (3, 2): 5, (2, 3): 5, (4, 3): -3, (3, 4): -3}

Negative Cycles Found


Vertix distance from source
----------------------------
Vertix 	 	 Distance
------ 	 	 --------
  A 		   -2
  B 		   -3
  C 		   2
  D 		   0
  E 		   -3
